In [ ]:
from PyLTSpice.LTSpiceBatch import SimCommander
from PyLTSpice.LTSpice_RawRead import LTSpiceRawRead
import ltspice
import os
import numpy as np
import time
import random
import itertools
from gym import Env
from gym.spaces import Discrete, Box
import matplotlib.pyplot as plt
from time import perf_counter

In [ ]:
observation_space = Box(np.array([1,1,1,1,1]), np.array([20,20,10,10,10]),dtype='float64')

C:\Users\TI_ML\anaconda3\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:
def netlist_manipulate(LTC,prev_width,curr_width):
    w1p,w3p,w4p,w6p,w7p = prev_width[0],prev_width[1],prev_width[2],prev_width[3],prev_width[4]
    w1,w3,w4,w6,w7 = curr_width[0],curr_width[1],curr_width[2],curr_width[3],curr_width[4]
    w2p=w1p
    w5p=w4p
    w8p=w6p
    w2=w1
    w5=w4
    w8=w6
    #print(LTC.netlist)
    LTC.remove_instruction('M1 VDD N001 N001 VDD CMOSP l=500n w={}u'.format(str(w1p)))  #w1
    LTC.remove_instruction('M2 VDD N001 N002 VDD CMOSP l=500n w={}u'.format(str(w2p)))
    LTC.remove_instruction('M3 VDD N002 vout VDD CMOSP l=500n w={}u'.format(str(w3p))) #w3
    LTC.remove_instruction('M4 N001 in1 N003 0 CMOSN l=500n w={}u'.format(str(w4p))) #w4
    LTC.remove_instruction('M5 N002 in2 N003 0 CMOSN l=500n w={}u'.format(str(w5p)))
    LTC.remove_instruction('M6 N004 N005 VSS 0 CMOSN l=500n w={}u'.format(str(w6p))) #w6
    LTC.remove_instruction('M8 vout N005 VSS 0 CMOSN l=500n w={}u'.format(str(w7p))) #w7
    LTC.remove_instruction('M9 N003 N005 VSS 0 CMOSN l=500n w={}u'.format(str(w8p)))
        ###################################################################
    ##############################################################
    c1=' w='+str(w1)+'u'
    c2=' w='+str(w2)+'u'
    c3=' w='+str(w3)+'u'Gain (dB)
    c4=' w='+str(w4)+'u'
    c5=' w='+str(w5)+'u'
    c6=' w='+str(w6)+'u'
    c8=' w='+str(w7)+'u'
    c9=' w='+str(w8)+'u'
   
    d1='M1 VDD N001 N001 VDD CMOSP'
    d2='M2 VDD N001 N002 VDD CMOSP'
    d3='M3 VDD N002 vout VDD CMOSP'
    d4='M4 N001 in1 N003 0 CMOSN'
    d5='M5 N002 in2 N003 0 CMOSN'
    d6='M6 N004 N005 VSS 0 CMOSN'
    d8='M8 vout N005 VSS 0 CMOSN'
    d9='M9 N003 N005 VSS 0 CMOSN'

    c=' l=500n'
    e1=d1+c+c1
    e2=d2+c+c2
    e3=d3+c+c3
    e4=d4+c+c4
    e5=d5+c+c5
    e6=d6+c+c6
    e8=d8+c+c8
    e9=d9+c+c9
    LTC.add_instructions(e1,e2,e3,e4,e5,e6,e8,e9,'.ac dec 100 1 1G')
    return LTC

In [ ]:
def Sim(lst):
    res=[]
    lst.append([7,87,3,6,37.5])
    lst=lst[::-1]
    temp=[]
    #print(lst)
    #AbsPath = os.path.dirname(os.path.realpath(__file__))
    LTC = SimCommander("2stage_opamp.asc",parallel_sims=100)
    for i in range(1,len(lst)):
        LTC = netlist_manipulate(LTC,lst[i-1],lst[i])
        LTC.run(wait_resource=False)
        LTC.wait_completion()
        l = ltspice.Ltspice('2stage_opamp_{}.raw'.format(str(i)))
        l.parse()
        freq = l.get_data('frequency')
        vout = l.get_data('V(vout)')
        k1=np.array(vout.data)
        k2 = np.degrees(np.angle(k1))
        k3=20*np.log10(np.abs(k1))
        k5=np.where((k3 >0.9) & (k3 < 1.2))
        print('GAIN = %f dB'%(k3[0]))
        if k3[0] > 0:
            res.append(k3[0])
        else:
            temp.append(lst[i])
    for i in temp:
        lst.remove(i)
    return res[::-1],lst

In [ ]:
'''
1. Sample n different parameter tuples from the obs.  space and store it in a list
2. Append the initial parameter tuple with the list
3. Create Netlist and generate empty raw files
4. Pick each tuple from the list and manipulate the netlist.
5. Run simulations parallelly
6. Write the outputs in the raw files
'''

In [ ]:
times=[]
x=[10,20,30,40,50,60,70,80,90,100]
for j in x:
    t1_start = perf_counter()
    for i in range(0,j):
        items.append(list(observation_space.sample()))
    fom,items=Sim(items)
    t1_stop = perf_counter()
    times.append(t1_stop-t1_start)

In [ ]:
print(times)

SEQUENTIAL

In [ ]:
def Sim1(w1,w3,w4,w6,w7):
    #AbsPath = os.path.dirname(os.path.realpath(__file__))
    w2=w1
    w5=w4
    w8=w6
    LTC = SimCommander("2stage_opamp.asc")
    # w1=' w='+str(w)
    # w2='M4 N001 in1 N003 0 CMOSN'
    # w3=' l=500n'
    # LTC.remove_instruction('M4 N001 in1 N003 0 CMOSN l=500n w=3u')
    # w4=w2+w1+w3
    # print(w4)

    LTC.remove_instruction('M1 VDD N001 N001 VDD CMOSP l=500n w=7u')  #w1
    LTC.remove_instruction('M2 VDD N001 N002 VDD CMOSP l=500n w=7u')
    LTC.remove_instruction('M3 VDD N002 vout VDD CMOSP l=500n w=87u') #w3
    LTC.remove_instruction('M4 N001 in1 N003 0 CMOSN l=500n w=3u') #w4
    LTC.remove_instruction('M5 N002 in2 N003 0 CMOSN l=500n w=3u')
    LTC.remove_instruction('M6 N004 N005 VSS 0 CMOSN l=500n w=6u') #w6
    LTC.remove_instruction('M8 vout N005 VSS 0 CMOSN l=500n w=37.5u') #w7
    LTC.remove_instruction('M9 N003 N005 VSS 0 CMOSN l=500n w=6u')
        ###################################################################

    ##############################################################
    c1=' w='+str(w1)+'u'
    c2=' w='+str(w2)+'u'
    c3=' w='+str(w3)+'u'
    c4=' w='+str(w4)+'u'
    c5=' w='+str(w5)+'u'
    c6=' w='+str(w6)+'u'
    c8=' w='+str(w7)+'u'
    c9=' w='+str(w8)+'u'

    ##################################################################
    
    
    d1='M1 VDD N001 N001 VDD CMOSP'
    d2='M2 VDD N001 N002 VDD CMOSP'
    d3='M3 VDD N002 vout VDD CMOSP'
    d4='M4 N001 in1 N003 0 CMOSN'
    d5='M5 N002 in2 N003 0 CMOSN'
    d6='M6 N004 N005 VSS 0 CMOSN'
    d8='M8 vout N005 VSS 0 CMOSN'
    d9='M9 N003 N005 VSS 0 CMOSN'

    
    ##################################################################
    c=' l=500n'
    ##################################################################
    
    e1=d1+c+c1
    e2=d2+c+c2
    e3=d3+c+c3
    e4=d4+c+c4
    e5=d5+c+c5
    e6=d6+c+c6
    e8=d8+c+c8
    e9=d9+c+c9
##################################################################
    LTC.add_instructions(e1,e2,e3,e4,e5,e6,e8,e9,'.ac dec 100 1 1G')

    ##################################################################


    # LTC.add_instructions(w4,'.ac dec 100 1 1G')
    #print(LTC.netlist)
    LTC.run(wait_resource=False)
    LTC.wait_completion()

    l = ltspice.Ltspice('2stage_opamp_1.raw') 
    # Make sure that the .raw file is located in the correct path
    l.parse() 


    freq = l.get_data('frequency')
    # V_out = l.get_data('V(out)')
    vout = l.get_data('V(vout)')

    k1=np.array(vout.data)
    k2 = np.degrees(np.angle(k1))

    k3=20*np.log10(np.abs(k1))
    k5=np.where((k3 >0.9) & (k3 < 1.2))
    #k6=np.real(freq[k5[0]])
    #print(k6)
    print('GAIN = %f dB'%(k3[0]))
    #print('GBW = %d MHz'%(k6[0].real/1000000))
    return k3[0]

In [ ]:
times=[]
x=[10,20,30,40,50,60,70,80,90,100]
for j in x:
    t1_start = perf_counter()
    for i in range(0,j):
        items.append(list(observation_space.sample()))
    for i in items:
        fom,items=Sim(i[0],i[1],i[2],i[3],i[4])
    t1_stop = perf_counter()
    times.append(t1_stop-t1_start)

In [ ]:
print(times)